# NLP Lab4 

## Work to do and assessment policy:

- The parts of this lab are independent.
- You may work by pairs, or alone. However, there is no benefit, nor increase of grade in case you work alone.
- Simply fill this notebook and upload it to [mvproxy](https://mvproxy.esiee.fr) no later than december 21st 23:59

Please first indicate below who you have worked with :

### <font color="red">Your answer here</font>

## Part A : Recurrent networks ($\thickapprox$ 8 pts)

### Introduction

In [TD3](https://perso.esiee.fr/~hilairex/5I-SI5/TD3-corr.pdf), we saw that a linear Elman network, used in teacher forcing mode, could properly learn the time series $x_t = \exp(at)$. This section only intends to demonstrate how Keras' [SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/) class operates on this elementary example.

Let's say we have a sequence of length 30, which is a realization of $\exp(at)_{t=1,...,30}$ for some $a \not= 0$.  We first generate the time series, then compute its image through the SimpleRNN layer:


In [1]:
# Exercice 1 du TD3 : apprendre un Elman sur exp(at)


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam

# Paramètres
a = 0.1  # Paramètre de la série
sequence_length = 30  # Longueur de la séquence
t = np.arange(sequence_length)
x_t = np.exp(a * t)  # Série temporelle x_t = exp(a * t)

# Préparation des données
X, y = [], []
for i in range(len(x_t) - 1):
    X.append(x_t[i:i+1])  # Utilisation de la valeur actuelle pour prédire la suivante
    y.append(x_t[i+1])

X = np.array(X)
y = np.array(y)

hidden= SimpleRNN(1, input_shape=(1, ), activation='linear', return_sequences=True, return_state=False)  # Activation linéaire
hidden(X.reshape(1,29,1))



c:\Users\afoua\miniconda3\envs\pynlp\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


<tf.Tensor: shape=(1, 29, 1), dtype=float32, numpy=
array([[[  1.3058513],
        [  2.7490404],
        [  4.344011 ],
        [  6.1067257],
        [  8.054827 ],
        [ 10.207811 ],
        [ 12.587228 ],
        [ 15.216889 ],
        [ 18.123116 ],
        [ 21.334991 ],
        [ 24.884663 ],
        [ 28.807657 ],
        [ 33.143238 ],
        [ 37.934795 ],
        [ 43.230286 ],
        [ 49.082706 ],
        [ 55.55063  ],
        [ 62.69879  ],
        [ 70.59873  ],
        [ 79.32951  ],
        [ 88.97852  ],
        [ 99.64233  ],
        [111.42765  ],
        [124.45245  ],
        [138.84708  ],
        [154.7556   ],
        [172.33723  ],
        [191.76794  ],
        [213.2422   ]]], dtype=float32)>

Because we have set <tt>return_sequences=True</tt> in the constructor of <tt>SimpleRNN</tt>, the value of the hidden state is returned at every time step. In addition, the <tt>return_state parameter</tt> controls whether the final state of the hidden state should also be returned or not. Hence, "return_sequences=True, return_state=False" corresponds to a many-to-many case, whereas "return_sequences=False, return_state=False" to a many-to-one case.

In [15]:
hidden= SimpleRNN(1, input_shape=(1, ), activation='linear', return_sequences=False, return_state=False)
hidden(X.reshape(1,29,1))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[249.31877]], dtype=float32)>

Note that <tt>SimpleRNN</tt> only implements the hidden state equation of Elman's model, not its output equation. To have the complete model, we must also add a Dense layer, which we do in the next snippets. In addition, $h_0$, the initial state, is *not* learnable by SimpleRNN. It can be specified when calling the layer by setting the initial_state parameter to some value or variable -- namely 1.

Here is the complete model for the many-to-many case :

In [2]:
inputs = Input(shape=(29, 1))

rnn_layer = SimpleRNN(1, use_bias=False, activation='linear', return_sequences=True, kernel_initializer='ones', recurrent_initializer='identity')

# lambda pour créer dynamiquement h0 selon la taille du lot à traiter
x = Lambda(lambda x: rnn_layer(x, initial_state=tf.ones((tf.shape(x)[0], 1))))(inputs)


outputs = Dense(1, use_bias=False, activation='linear')(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

# L'essayer
model(X.reshape(1,29,1))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 29, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 29, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 29, 1)          │             1 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1 (4.00 B)

 Trainable params: 1 (4.00 B)

 Non-trainable params: 0 (0.00 B)

<tf.Tensor: shape=(1, 29, 1), dtype=float32, numpy=
array([[[  -3.3692932],
        [  -5.231116 ],
        [  -7.288748 ],
        [  -9.562783 ],
        [ -12.07598  ],
        [ -14.853493 ],
        [ -17.923119 ],
        [ -21.31558  ],
        [ -25.06483  ],
        [ -29.208391 ],
        [ -33.787735 ],
        [ -38.848694 ],
        [ -44.441917 ],
        [ -50.623386 ],
        [ -57.45496  ],
        [ -65.00503  ],
        [ -73.34914  ],
        [ -82.57081  ],
        [ -92.76233  ],
        [-104.025696 ],
        [-116.47364  ],
        [-130.23074  ],
        [-145.43471  ],
        [-162.23769  ],
        [-180.80785  ],
        [-201.33104  ],
        [-224.01268  ],
        [-249.07977  ],
        [-276.78317  ]]], dtype=float32)>

Notice that there is just one trainable parameter, as expected. Then, we can train the model:

In [ ]:
# Entraînement du modèle
model.fit(X.reshape(1, 29, 1), y.reshape(1,29,1), epochs=2000, verbose=0)

# Prédiction
predictions = model.predict(X.reshape(1, 29, 1))

# Affichage des résultats
print("Valeurs réelles :", y)
print("Prédictions :", predictions.flatten())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Valeurs réelles : [ 1.10517092  1.22140276  1.34985881  1.4918247   1.64872127  1.8221188
  2.01375271  2.22554093  2.45960311  2.71828183  3.00416602  3.32011692
  3.66929667  4.05519997  4.48168907  4.95303242  5.47394739  6.04964746
  6.68589444  7.3890561   8.16616991  9.0250135   9.97418245 11.02317638
 12.18249396 13.46373804 14.87973172 16.44464677 18.17414537]
Prédictions : [ -0.48094475  -0.74670786  -1.0404215   -1.3650253   -1.7237679
  -2.1202397   -2.558409    -3.042661    -3.577842    -4.1693087
  -4.82298     -5.5453987   -6.3437953   -7.22616     -8.2013235
  -9.279046   -10.470114   -11.786447   -13.24122    -14.848993
 -16.625856   -18.589594   -20.759861   -23.158377   -25.809147
 -28.7387     -31.976357   -35.55452    -39.509003  ]


### Problem statement

Let $\Sigma=\lbrace \lbrace, \rbrace, a, b, \rbrace$ be an alphabet of only 4 characters. We consider strings made from $\Sigma$ that have the following property : whenever a '{' character appears, it must match a corresponding '}' character later on, without that any new '{' character may appear in the mean time. In other words, our strings obey the regex
```pre
({[ab]*})*
```

### Question A1
Propose an Elman network that solves this recognition problem *exactly* : it must achieve a 100% accuracy whatever the input source. For your convenience :
* it is suggested that $\lbrace = (1,0,0,0)$, $\rbrace=(0,1,0,0)$, $a=b=(0,0,1,0)$ as a starter, but <b>you may choose different dimensions and values</b>.
* Elman's equations are given as $$ \begin{align*} h_t &= f(U h_{t-1} + Wx_t) \\ y_t&=g(Vh_t) \end{align*}$$
* and dummy expressions of $U,V,W$ are given below as your answer, so that you can use a Latex template.

This question does not require any programming at all : to answer, it is sufficient to fully specify all matrices, and ideally add 1-2 sentences exlaining how your network works. Beware : invalid strings must be <tt>rejected</tt>, not accepted.

### <font color="red">Your answer here</font>

### Réponse A1

Pour reconnaître le langage `({[ab]*})*`, j’ai construit un petit automate à 3 états, puis je l’ai transposé dans un réseau d’Elman.

**États (codés en one-hot) :**
- \(q_0 = (1,0,0)^T\) : on est en dehors d’un bloc `{}`.
- \(q_1 = (0,1,0)^T\) : on est à l’intérieur du bloc.
- \(q_{\text{err}} = (0,0,1)^T\) : erreur (état absorbant).

État initial : \(h_0 = (1,0,0)^T\).

**Encodage des symboles :**
- `{` = (1,0,0,0)
- `}` = (0,1,0,0)
- `a` et `b` = (0,0,1,0)

**Transitions voulues :**
- Depuis \(q_0\) : `{` → \(q_1\), sinon erreur.
- Depuis \(q_1\) : `a`/`b` → rester dans \(q_1\), `}` → retourner à \(q_0\), `{` → erreur.
- Depuis \(q_{\text{err}}\) : on reste dedans.

Pour reproduire ça avec un Elman, j’utilise :
\[
h_t = H(Uh_{t-1} + Wx_t)
\]
avec \(H\) une fonction de seuil, ce qui permet d’obtenir un vecteur one-hot correspondant à l’état suivant. Les matrices \(U\) et \(W\) sont choisies pour rendre positives uniquement les transitions valides (et négatives sinon), ce qui “simule” l’automate.

Enfin, pour décider si la chaîne est valide, je regarde seulement l’état final :
- si on finit en \(q_0\) → chaîne acceptée,
- si on finit en \(q_1\) ou \(q_{\text{err}}\) → rejet.

Une sortie simple est par exemple :
\[
y = \sigma(Vh_T), \quad V = (1, 0, -1)
\]
ce qui donne une valeur proche de 1 quand la chaîne est correcte.

Ce réseau reconnaît donc exactement les chaînes du langage demandé.


We think that $U=\left( \begin{array}{cccc}
1 & 2 & 3 & 4 \\
5 & 6 & 7 & 8 \\
9 & 0 & 1 & 2 \\
3 & 4 & 5 & 6
\end{array}
\right)$, and $W=\left( \begin{array}{cccc}
1 & 2 & 3 & 4 \\
5 & 6 & 7 & 8 \\
9 & 0 & 1 & 2 \\
3 & 4 & 5 & 6
\end{array}
\right)$, and $V= (1,2,3,4)^\top$ are clever choices, because our network would then operate as follows :
* if input = '{', then
* else if input = '}', then
* ...

### Question A2

The following code trains an LSTM network to properly separate the strings you have just worked on. It should achieve a very high accuracy (nearby 100%). Try to interpret the matrices it prints, in a broad sense : which coefficients are likely to trigger which gate, when which symbol is met ?. If you can't, explain how LSTM could be used to sucessfully process such strings, even if their length become arbitrary large (once again, this amounts to determining which gate should react to which symbol, and why).

This question does not require any programming.

### <font color="red">Your answer here</font>


### Réponse A2

Le code entraîne un LSTM pour reconnaître les mêmes chaînes que dans la question A1 (le langage `({[ab]*})*`). On obtient des matrices de poids pour les quatre portes du LSTM : entrée (W_i / U_i), oubli (W_f / U_f), candidat (W_c / U_c) et sortie (W_o / U_o).

Franchement, lire chaque coefficient un par un n’a pas trop de sens, car les valeurs dépendent beaucoup de l’init et de l’entraînement. Par contre, on peut expliquer **en gros** ce qui se passe et quel type de comportements ces matrices apprennent.

---

#### 1. Ce que les neurones doivent mémoriser

Pour ce langage, le LSTM doit essentiellement savoir deux choses :

1. Est-ce qu’on est **dans un bloc** `{ ... }` ou **en dehors** ?
2. Est-ce qu’on a déjà rencontré une **erreur** (caractère interdit, `}` sans `{`, `{` imbriqué, bloc jamais refermé, etc.) ?

On peut imaginer qu’un ou deux neurones de la cellule mémoire servent à coder “on est dans un bloc” (valeur forte quand on est entre `{` et `}`), et qu’un autre neurone encode qu’on est passé en “mode erreur”.

---

#### 2. Rôle des différentes portes sur les symboles `{`, `}`, `a`, `b`

Sans analyser tous les nombres, on peut raisonner comme suit :

- **Quand le LSTM lit `{` alors qu’il était en dehors** :
  - les poids associés au symbole `{` dans la **input gate** \(W_i\) et dans le **candidat** \(W_c\) sont configurés pour **ouvrir** le bloc : on écrit dans la cellule mémoire une valeur indiquant “on est maintenant à l’intérieur” ;
  - la **forget gate** \(W_f\) peut aussi remettre à zéro d’anciennes infos inutiles.

- **Quand il lit `a` ou `b` à l’intérieur d’un bloc** :
  - les poids correspondants dans \(W_f\) sont en général proches de 1 (porte d’oubli ouverte) pour **garder** la mémoire “on est dans un bloc” ;
  - \(W_i\) et \(W_c\) ne changent pas beaucoup la cellule : on reste dans le même état.

- **Quand il lit `}` à la fin d’un bloc** :
  - la porte d’oubli \(f_t\) et le candidat \(\tilde{c}_t\) sont réglés pour **faire redescendre** la valeur qui codait “dans un bloc” et revenir à un état “extérieur” ;
  - si `}` arrive alors qu’on était dehors, d’autres neurones/canaux peuvent passer dans un état “erreur” que la porte de sortie fera ressortir.

- **En cas de pattern impossible** (par exemple `{` à l’intérieur d’un bloc, ou des lettres en dehors des blocs) :
  - certaines combinaisons de poids dans \(W_i, W_c\) vont écrire une valeur spéciale dans la cellule (par exemple quelque chose de très négatif ou positif) ;
  - la porte d’oubli \(f_t\) peut fermer l’ancienne info pour ne garder que l’état “erreur” ;
  - une fois que cet état erreur est écrit, la **output gate** \(o_t\) laisse sortir principalement cette info, et le réseau rejettera la chaîne à la fin.

En résumé, certains coefficients “grands en valeur absolue” dans les lignes associées à `{`, `}`, `a`, `b` vont surtout contrôler quelles portes s’ouvrent pour quels symboles, et donc comment la mémoire se met à jour.

---

#### 3. Pourquoi ça marche même pour des chaînes longues

L’intérêt du LSTM, par rapport à un simple RNN, est que la **cellule mémoire** \(c_t\) peut garder une information sur une très longue séquence grâce à la porte d’oubli :

- tant que la séquence reste correcte, les portes sont réglées pour **conserver** la bonne info (“dans un bloc” / “dehors” / “erreur ou pas”) ;
- la mise à jour de \(c_t\) dépend juste de \(c_{t-1}\) et du symbole courant, donc la longueur de la chaîne n’est pas un problème : on applique toujours la même règle à chaque pas de temps.

Une fois les bons motifs appris (quelles portes réagissent à `{`, `}`, `a`, `b`), le LSTM se comporte en pratique comme un **automate fini** implémenté de façon continue. C’est pour ça qu’il arrive à séparer correctement les chaînes valides des invalides, et que l’accuracy sur le test est proche de 100 %.

En conclusion, même si on ne “lit” pas tous les coefficients, on peut dire que :
- certains poids associés à `{` déclenchent l’ouverture d’un bloc,
- ceux associés à `}` déclenchent la fermeture (ou l’erreur),
- ceux associés à `a`/`b` maintiennent l’état interne,
et les différentes portes (input, forget, output) coopèrent pour que la mémoire se comporte comme l’automate de la question A1.



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Fixer la graine pour la reproductibilité (les résultats varient peu, mais c'est une bonne pratique)
tf.random.set_seed(42)

# --- 1. Paramètres et Préparation des Données ---

CHARS = {'{': 0, '}': 1, 'a': 2, 'b': 3}
VOCAB_SIZE = len(CHARS) # 4
MAX_LEN = 10
# Nous gardons la même dimension pour l'état caché (h_t et c_t)
HIDDEN_DIM = 4 

def generate_sequences(num_sequences):
    """Génère des chaînes valides (1) et invalides (0) selon ({[ab]*})+."""
    X_list = []
    y_list = []
    
    while len(X_list) < num_sequences:
        # --- Séquence Valide (Label 1) ---
        valid_seq = ""
        num_blocks = np.random.randint(1, 4) 
        
        for _ in range(num_blocks):
            max_content_len = MAX_LEN - len(valid_seq) - 2
            if max_content_len < 0: break
            
            content_len = np.random.randint(0, max_content_len + 1)
            content = ''.join(np.random.choice(['a', 'b'], content_len))
            valid_seq += '{' + content + '}'
        
        if len(valid_seq) > 0 and len(valid_seq) <= MAX_LEN:
            X_list.append(valid_seq.ljust(MAX_LEN, ' '))
            y_list.append(1) 
        
        if len(X_list) >= num_sequences: break

        # --- Séquence Invalide (Label 0) ---
        invalid_type = np.random.randint(0, 4)
        invalid_seq = ""
        
        if invalid_type == 0:
            content_len = np.random.randint(0, 8)
            content = ''.join(np.random.choice(['a', 'b'], content_len))
            invalid_seq = '{' + content
        elif invalid_type == 1:
            invalid_seq = '}' + ''.join(np.random.choice(['{', 'a', 'b'], 2))
        elif invalid_type == 2:
            invalid_seq = 'a{b}'
        else:
            invalid_seq = '{' + '}' + 'a'
            
        if len(invalid_seq) > 0 and len(invalid_seq) <= MAX_LEN:
            X_list.append(invalid_seq.ljust(MAX_LEN, ' '))
            y_list.append(0) 

        if len(X_list) >= num_sequences: break

    return X_list, np.array(y_list)

def encode_sequences(sequences):
    """Encode les chaînes en tenseurs One-Hot."""
    X_encoded = np.zeros((len(sequences), MAX_LEN, VOCAB_SIZE), dtype=np.float32)
    
    for i, seq in enumerate(sequences):
        for t, char in enumerate(seq):
            if char in CHARS:
                X_encoded[i, t, CHARS[char]] = 1.0

    return X_encoded

N_SAMPLES = 5000
X_raw, y = generate_sequences(N_SAMPLES)
X_data = encode_sequences(X_raw)

split_idx = int(0.9 * N_SAMPLES)
X_train, X_test = X_data[:split_idx], X_data[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

y_train_oh = to_categorical(y_train, num_classes=2)
y_test_oh = to_categorical(y_test, num_classes=2)

print(f"Forme des données d'entraînement : X={X_train.shape}, y={y_train_oh.shape}")


# --- 2. Construction et Entraînement du Modèle LSTM ---

model = Sequential([
    LSTM(
        HIDDEN_DIM, 
        input_shape=(MAX_LEN, VOCAB_SIZE),
        return_sequences=False
    ),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\nDébut de l'entraînement de l'architecture LSTM...")
history = model.fit(
    X_train, y_train_oh, 
    epochs=50, 
    batch_size=32, 
    validation_data=(X_test, y_test_oh),
    verbose=0
)

loss, accuracy = model.evaluate(X_test, y_test_oh, verbose=0)
print(f"\nPrécision sur l'ensemble de test (LSTM): {accuracy * 100:.2f}%")


# --- 3. Extraction et Examen des Matrices de Poids ---

lstm_layer = model.layers[0]
weights = lstm_layer.get_weights()

W = weights[0] # Matrice d'entrée complète (Input Kernel W) - Shape (4, 16)
U = weights[1] # Matrice récurrente complète (Recurrent Kernel U) - Shape (4, 16)
# B = weights[2] # Biais (Bias) - Shape (16,)

# Keras/TensorFlow stocke les poids dans l'ordre: [i, f, c, o]
# Nous devons découper W (4x16) et U (4x16) en 4 matrices (4x4)
slice_size = HIDDEN_DIM # 4

# Slicing pour W (Poids d'Entrée)
W_i = W[:, 0*slice_size:1*slice_size] # Input Gate
W_f = W[:, 1*slice_size:2*slice_size] # Forget Gate
W_c = W[:, 2*slice_size:3*slice_size] # Candidate Cell State
W_o = W[:, 3*slice_size:4*slice_size] # Output Gate

# Slicing pour U (Poids Récurrents)
U_i = U[:, 0*slice_size:1*slice_size]
U_f = U[:, 1*slice_size:2*slice_size]
U_c = U[:, 2*slice_size:3*slice_size]
U_o = U[:, 3*slice_size:4*slice_size]


print("\n" + "="*80)
print("ANALYSE DES MATRICES DE POIDS D'ENTRÉE LSTM (W_i, W_f, W_c, W_o)")
print("Cols: Entrées ({, }, a, b) / Lignes: Neurones Cachés (h1 à h4)")
print("="*80)

char_indices = ['{', '}', 'a', 'b']
hidden_indices = ['h1', 'h2', 'h3', 'h4']

def print_matrix(name, matrix):
    print(f"\n--- {name} ---")
    print(f"{'':>6} {'{':>8} {'}':>8} {'a':>8} {'b':>8}")
    for i in range(HIDDEN_DIM):
        print(f"{hidden_indices[i]}: {matrix[i, 0]:8.4f} {matrix[i, 1]:8.4f} {matrix[i, 2]:8.4f} {matrix[i, 3]:8.4f}")

print_matrix("Matrice d'Entrée W_i (Input Gate)", W_i)
print_matrix("Matrice d'Oubli W_f (Forget Gate)", W_f)
print_matrix("Matrice d'État Candidat W_c (Candidate Cell State)", W_c)
print_matrix("Matrice de Sortie W_o (Output Gate)", W_o)

print("\n" + "="*80)
print("ANALYSE DES MATRICES DE POIDS RÉCURRENTS LSTM (U_i, U_f, U_c, U_o)")
print("Cols/Lignes: Neurones Cachés (h1 à h4)")
print("="*80)

def print_recurrent_matrix(name, matrix):
    print(f"\n--- {name} ---")
    print(f"{'':>6} {'h1':>8} {'h2':>8} {'h3':>8} {'h4':>8}")
    for i in range(HIDDEN_DIM):
        print(f"{hidden_indices[i]}: {matrix[i, 0]:8.4f} {matrix[i, 1]:8.4f} {matrix[i, 2]:8.4f} {matrix[i, 3]:8.4f}")

print_recurrent_matrix("Matrice d'Entrée U_i", U_i)
print_recurrent_matrix("Matrice d'Oubli U_f", U_f)
print_recurrent_matrix("Matrice d'État Candidat U_c", U_c)
print_recurrent_matrix("Matrice de Sortie U_o", U_o)


Forme des données d'entraînement : X=(4500, 10, 4), y=(4500, 2)

Début de l'entraînement de l'architecture LSTM...

Précision sur l'ensemble de test (LSTM): 100.00%

ANALYSE DES MATRICES DE POIDS D'ENTRÉE LSTM (W_i, W_f, W_c, W_o)
Cols: Entrées ({, }, a, b) / Lignes: Neurones Cachés (h1 à h4)

--- Matrice d'Entrée W_i (Input Gate) ---
              {        }        a        b
h1:  -0.1381  -0.2418  -0.8643   0.8981
h2:   0.7994   0.6667   0.1277   0.6879
h3:  -0.2690   0.6836  -0.4907   0.1601
h4:   0.3706   0.2812   0.3868  -0.0256

--- Matrice d'Oubli W_f (Forget Gate) ---
              {        }        a        b
h1:   0.1378  -1.4328  -0.1523   0.4121
h2:   2.1547   1.2244  -1.5588   1.5166
h3:   0.5037  -0.0785  -0.9692  -0.1075
h4:   0.0680  -0.1215  -0.9417  -0.2530

--- Matrice d'État Candidat W_c (Candidate Cell State) ---
              {        }        a        b
h1:   0.3200  -0.4103  -0.2409  -0.5035
h2:  -0.1351  -0.2555   1.3123  -0.0370
h3:   0.6379  -0.5162  -0.1658 

## Part B : text classification ($\thickapprox$ 6 pts)

In this part, you will have to achieve the implementation of the RNN-based model shown on slide 28 of [Chapter 4](https://perso.esiee.fr/~hilairex/5I-SI5/rnn.pdf). The network accepts words as input, from sentences which don't exceed a certain length, and aim to perform text classification. 

You will work on the the IMDB reviews dataset, hosted by Kaggle [here](https://www.kaggle.com/code/trentpark/data-analysis-basics-imdb-dataset). The reviews have two outcomes : positive, or negative. A copy of this dataset can be found at https://mvproxy.esiee.fr/NLP/Lab4

The following code snippets perform the first steps on text for you - loading, vectorising, and training a basic (non-recurrent) FFN.

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import tensorflow as tf
from keras.models import Sequential

reviews = pd.read_csv("IMDB Dataset.csv")
reviews.head(2)

ModuleNotFoundError: No module named 'pandas'

We first perform a standard test/train split. During development, I strongly suggest that you first use a small amount of samples (1000) for validation. IMDb has 50000 reviews, which is too much. Keep in ming that training RNNs is *slow*

In [5]:
train, test= train_test_split(reviews, shuffle=True, train_size=1000, test_size=200)

The next step is to vectorize the text. In Lab3, I provided a vecto() function which did this, with relevant padding. I also mentioned Keras offered a TextVectorization layer which did exactly the same job. Its effects are shown below. 

In particular, note that unknown words yield an index of 1, and 0 is used for padding. So real indexation starts at index 2.

In [6]:
# text vectorization : quick demo
vecto= tf.keras.layers.TextVectorization(max_tokens=99, output_mode='int', output_sequence_length=10)
vecto.adapt([["I am the king of the world"],["You are the queen"]])
vecto([["I am the queen"],["World is king unknown"]])

<tf.Tensor: shape=(2, 10), dtype=int64, numpy=
array([[ 8, 10,  2,  5,  0,  0,  0,  0,  0,  0],
       [ 4,  1,  7,  1,  0,  0,  0,  0,  0,  0]])>

We now change the call to adapt the layer to our train data. Note that IMDb reviews are rather long (about 300 words / review on average)

In [7]:
max_words=20000  # the vocabulary size
seq_len=300     # maximum sequence length
vecto= tf.keras.layers.TextVectorization(max_tokens=max_words, output_mode='int', output_sequence_length=300)
vecto.adapt(train['review'].to_list())


We are now ready to define our model. Below, I first demonstrate a model with input and vectorization layer alone .

In [8]:
# building model : vectorization alone
model= Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vecto)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()
model.predict(tf.constant([['I am the king']]))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_1            │ (None, 300)            │             0 │
│ (TextVectorization)             │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


array([[ 11, 260,   2, 614,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

As we saw in labs 2 and 3, embeddings are mandatory. Hence, we will add an Embedding layer, but as opposed as what we did before, we will not initialize if from LSA, nor put it constant. Instead, we will let the model optimize this layer, possibly using dropout (if you use the related option). 
The dimension of 80 below is a crude estimation (barely from lab2 and results on LSA)  

In [9]:
model= Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vecto)
model.add(tf.keras.layers.Embedding(max_words+2, 80, input_length=seq_len))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()
model.predict(tf.constant([['I am the king']]))


/home/hilairex/pynlp/lib64/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_1            │ (None, 300)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 300, 80)        │       240,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,160 (938.12 KB)

 Trainable params: 240,160 (938.12 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


array([[[ 0.01580559,  0.02355259,  0.02332291, ..., -0.004517  ,
          0.0277617 , -0.02131273],
        [ 0.01668474, -0.03429715, -0.04509323, ..., -0.00417768,
         -0.01822596,  0.0195891 ],
        [ 0.03996921, -0.02546388,  0.00366718, ...,  0.02894926,
         -0.00926412, -0.01020152],
        ...,
        [ 0.00812037, -0.04475567,  0.0241028 , ...,  0.04265859,
         -0.0465613 , -0.00206579],
        [ 0.00812037, -0.04475567,  0.0241028 , ...,  0.04265859,
         -0.0465613 , -0.00206579],
        [ 0.00812037, -0.04475567,  0.0241028 , ...,  0.04265859,
         -0.0465613 , -0.00206579]]], dtype=float32)

### Question B1

The following code reuses the previous ingredients up to TextVectorization, but lacks the definition of a model.

Fill the part marked as
```pre
## ....
# A vous de jouer: définissez ici le modèle conformément au cours
## ....
```
so that the model defined there conform to the one shown on slide 38 of Chapter 4 [here](https://perso.esiee.fr/~hilairex/AIC-5102B/rnn.pdf). To be properly summarized by the summary() function, your model should be built using Keras' *functional* API,  not the *sequential()* function.  The recurrent network should be a bi-directional LSTM. The output network may boil down to a single layer network with appropriate dimensions.

Some pieces of advice :
- Remember that embedding turns integer indexes into vectors. Hence your input data is a sequence of *vectors* whatever type of RNN you use.
- Be careful to overfitting and shapes.
- In the end, you want a single scalar to represent a decision : 1 or 0 (positive or negative)
- Keras has a [Bidirectional](https://keras.io/api/layers/recurrent_layers/bidirectional/) and a [Concatenate](https://keras.io/api/layers/merging_layers/concatenate/) layers, which can prove useful. However, you may build your model without using them, by resorting to Keras' functional API.

With only 5 epochs, your model should achieve a rather good accuracy (~ 85%), but it might take longer to converge, depending on initial conditions. 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences

# -------------------------------
# 1. Paramètres généraux
# -------------------------------

max_features = 20000   # Nombre max de mots dans le vocabulaire
maxlen = 300           # Longueur max des séquences (padding/tronquage)
batch_size = 32
embedding_dim = 128
rnn_units = 128
epochs = 5

# -------------------------------
# 2. Chargement du dataset IMDB (fichier CSV)
# -------------------------------

df = pd.read_csv("IMDB Dataset.csv")   # Colonnes : "review", "sentiment"

# Conversion des labels
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})

texts = df["review"].astype(str).values
labels = df["label"].values

print("Nombre d'exemples :", len(texts))
print("Exemple de texte :", texts[0][:200], "...")

# -------------------------------
# 3. Split train/test
# -------------------------------

from sklearn.model_selection import train_test_split
x_train_text, x_test_text, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# -------------------------------
# 4. TextVectorization
# -------------------------------

vectorizer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=maxlen
)

# Apprentissage du vocabulaire
vectorizer.adapt(x_train_text)

# Encodage en séquences d'entiers
x_train = vectorizer(x_train_text)
x_test  = vectorizer(x_test_text)

print("Forme x_train :", x_train.shape)
print("Forme x_test  :", x_test.shape)

# -------------------------------
# 5. Construction du modèle
# -------------------------------

inputs = tf.keras.layers.Input(shape=(maxlen,))

## ....
# A vous de jouer: définissez ici le modèle conformément au cours
## ....

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# -------------------------------
# 6. Entraînement du modèle
# -------------------------------

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=2
)

# -------------------------------
# 7. Évaluation sur le jeu de test
# -------------------------------

loss, acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\nPrécision sur le test : {acc * 100:.2f} %")


### Question B2

What happens if your code calls SimpleRNN() instead of LSTM() (all other things being equel)? Give a simple explanation of what is very likely to happen.

### <font color="red">Your answer here</font>



## Part C : inference in neural machine translation ($\thickapprox$ 8 pts)

In this part, you will work on the coder+decoder network shown on slide 26 of [Chapter 5](https://perso.esiee.fr/~hilairex/AIC-5102B/lstm.pdf) of the course. We will use this model to translate english sentences to french sentences. To keep computation times acceptable on a laptop, we will not perform translation word by word, as this would require vocabularies of about 50k words in each language; but character by character, which involves merely 100 tokens in each language. Shortly said, we are using seq2seq for character sequence translation.

The dataset we use for training is derived from transcripts of the European parliament - see https://www.statmt.org/europarl/ 
 
On [mvproxy](https://mvproxy.esiee.fr/NLP), you will find several files :
- english_filtered and french_filtered : reworked versions of english to french translations of sentences from Europarl.
- lab4c_train.py: Python implementation to train the network over the former files. This code is parameterized by lstm_units, which defines the dimension of the hidden space of the LSTM network used in both coder and decoder.
- en2fr<i>xxx</i>.keras + voc.pkl : the Keras model produced by training the model with <i>xxx</i> LSTM units, for both coder and decoder  + the related vocabularies found when parsing english_filtered and french_filtered
- lab4c_decode.py : simple implementation of greedy decoding from the decoder, still parameterized by lstm_units.

Note that we have separated the problem in two parts : 
- Training, or lab4c_train.py, which produces en2fr<i>xxx</i>.keras + voc.pkl the very first time it is ran. When ran again, it loads the vocabulary and last model it produced from disk, and refines training. If you run it again, the first training epochs should display the training and validation accuracies. All models have been trained using 2000 epochs.
- And decoding, which re-reads these files to rebuild the whole model. 

### Question C1

Run the decoder for some <i>xxx</i>, and test it using some sentences found in the source (english) file. You should normally find that your text is not properly decoded, but that the output is not completely random, and somewhat resembles some imaginary french language. In other words, the seq2seq models you are using are hallucinating. 

Assuming that what you did in part A can be generalized, and would be the only way for a RNN to properly distinguish strings (this is really an assumption): 
- are these hallucinations a surprise ?
- what should be changed so that the model you are using have chances not to hallicinate ?
  
Please provide a clear explanation of what can happen, considering all parameters (in particular, shapes of matrices), and also how decoding is done. You may simplify the problem and explain what happens if the model were an Elman network (extending to LSTM is just a matter of additional gates and matrices).

### Your answer here

### Question C2

Below is a copy of lab4c_decode.py, the code of the decoder. Change the code of this function so that it no longer uses greedy decoding, but a beam search decoding of depth 4. Your code should be implemented as is Algorithm 1 on slide 30 of [chapter 5](https://perso.esiee.fr/~hilairex/AIC-5102B/lstm.pdf), with k=4. It should output several candidate strings rather than a single one. Test it using 3 strings.

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Robust decoder that:
 - uses model.inputs[0] to get encoder input tensor
 - re-applies encoder Embedding before calling encoder LSTM
 - reuses decoder Embedding + LSTM + Dense for one-step decoding
 - performs greedy decoding
"""

import pickle
import sys
import numpy as np
from tensorflow import keras

units = 512
model_file = f"en2fr{units}.keras"
voc_file   = "voc.pkl"
begin = '\x02'
end   = '\x03'
max_target_len = 200

# -----------------------
# Load model and vocab
# -----------------------
print(f"Loading model: {model_file}")
model = keras.models.load_model(model_file)
model.summary()

with open(voc_file, 'rb') as f:
    voc, char2num, num2char = pickle.load(f)
print("Vocabulary loaded:", [len(v) for v in voc])


# Encoder
enc_emb_layer = model.get_layer('l_enc_embedding')
enc_lstm_layer = model.get_layer('l_enc_lstm')

# Decoder
dec_emb_layer = model.get_layer('l_dec_embedding')
dec_lstm_layer = model.get_layer('l_dec_lstm')
dec_dense_layer = model.get_layer('l_dec_dense')


# -----------------------
# Build encoder model: enc_input -> [h, c]
# -----------------------
# Apply encoder embedding to the encoder input tensor, then call the trained LSTM on it
enc_input_tensor = keras.Input(shape=(None,), dtype='int32')
enc_embedded = enc_emb_layer(enc_input_tensor)   # (batch, timesteps, emb_dim)
enc_out, enc_h, enc_c = enc_lstm_layer(enc_embedded)
encoder_model = keras.Model(inputs=enc_input_tensor, outputs=[enc_h, enc_c])
print("Encoder model built.")

# -----------------------
# Build decoder model for one-step decoding
# -----------------------
state_size = dec_lstm_layer.units

dec_state_input_h = keras.Input(shape=(state_size,), name='dec_state_input_h')
dec_state_input_c = keras.Input(shape=(state_size,), name='dec_state_input_c')
dec_input_single = keras.Input(shape=(1,), dtype='int32', name='dec_input_single')  # a single timestep token

# apply decoder embedding then decoder LSTM (one step)
dec_embedded = dec_emb_layer(dec_input_single)  # (batch, 1, emb_dim)
dec_outputs, dec_h, dec_c = dec_lstm_layer(dec_embedded, initial_state=[dec_state_input_h, dec_state_input_c])
dec_outputs = dec_dense_layer(dec_outputs)  # (batch, 1, vocab_size)

decoder_model = keras.Model(
    [dec_input_single, dec_state_input_h, dec_state_input_c],
    [dec_outputs, dec_h, dec_c]
)
print("Decoder model built.")

# -----------------------
# Utilities: encode input / greedy decode
# -----------------------
def encode_input(text: str):
    """Encode source text into int32 array (1, L). Append end marker if missing."""
    if not text.endswith(end):
        text = text + end
    arr = np.zeros((1, len(text)), dtype='int32')
    for i, ch in enumerate(text):
        arr[0, i] = char2num[0].get(ch, 0)
    return arr

def decode_greedy(src_text: str, maxlen: int = max_target_len):
    enc_seq = encode_input(src_text)
    # get encoder states
    h, c = encoder_model.predict(enc_seq, verbose=0)

    # start token (<begin>)
    cur_token = np.array([[char2num[1][begin]]], dtype='int32')
    decoded_chars = []

    for _ in range(maxlen):
        out_tokens, h, c = decoder_model.predict([cur_token, h, c], verbose=0)
        probs = out_tokens[0, 0, :]
        idx = int(np.argmax(probs))
        ch = num2char[1][idx]
        if ch == end:
            break
        decoded_chars.append(ch)
        cur_token = np.array([[idx]], dtype='int32')

    return "".join(decoded_chars)

# -----------------------
# Interactive loop
# -----------------------
print("Ready. Enter a source sentence (empty to quit).")
try:
    while True:
        s = input("Source > ").strip()
        if s == "":
            break
        translation = decode_greedy(s)
        print("→", translation)
except KeyboardInterrupt:
    print("\nExiting.")
    sys.exit(0)

